第一題:<br>
下載商工開放平台的公司統編csv ，選擇一類下載，再使用Python開發爬蟲且使用Proxy進入商工平台，輸入統一編號獲取公司登記地址，地址轉成經緯度座標(TGOS或第三方套件)，相關數據請使用ORM存到本機的 Sqlite 資料庫。 

1. 商工開放平台: https://data.gcis.nat.gov.tw/od/datacategory
2. 商工登記服務入口: https://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do
3. TGOS查詢入口: https://www.tgos.tw/tgos/Web/Address/TGOS_Address.aspx


In [ ]:
# !kill -9 -1
!sudo apt update
!sudo apt install chromium-chromedriver

!pip3 install selenium
!pip3 install twd97
!pip3 install peewee

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [96.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 k

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By as BY
from selenium.webdriver.support import ui as UI
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from google.colab import files
from peewee import *
import sqlite3
import twd97 as TWD97
import math
import csv
import os

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
def getFile(fileName):
  '''
  for loop get filename and gui
  '''
  gui_list = {}
  with open('./{}'.format(fileName), newline='') as csvfile:
      rows = csv.reader(csvfile)
      for row in rows:
          if len(row[0]) == 8 :
              gui_list[row[1]]=row[0]
  return gui_list

In [ ]:
def initSelenium(proxy):
  '''
  init selenium and setting options
  '''
  #selenium options setting
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("--disable-infobars")
  # add proxy
  if proxy != None:
    chrome_options.add_argument('--proxy-server=http://{}'.format(proxy))

  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  #set browser loading timeout
  driver.set_page_load_timeout(120)

  return driver

In [ ]:
def getAddr(driver,gui):
  '''
  get company address from web
  '''
  driver.get("https://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do")
  driver.find_element_by_xpath("//input[@name='qryCond']").send_keys(gui)
  driver.find_element_by_id('qryBtn').click()
  UI.WebDriverWait(driver,15).until(lambda driver: driver.find_element_by_class_name('companyName'))
  # get company address
  address = driver.find_element_by_xpath("//div[@class='panel-heading companyName']/following-sibling::div[1]").text.split('地址：')[1].split(' , ')[0]
  return  driver,address

In [ ]:
def getTWD97(driver,address):
  '''
  get TWD97 use address from TGOSweb
  '''
  driver.get("https://map.tgos.tw/TGOSCloud/Web/Map/TGOSViewer_Map.aspx?addr={}".format(address))
  UI.WebDriverWait(driver,15).until(lambda driver: driver.find_element_by_xpath("//a[@class='selectS messageBox-RoadPath']/parent::div"))
  # address to TWD97
  x = float(driver.find_element_by_xpath("//a[@class='selectS messageBox-RoadPath']/parent::div").get_attribute('data-px'))
  y = float(driver.find_element_by_xpath("//a[@class='selectS messageBox-RoadPath']/parent::div").get_attribute('data-py'))
  return driver,(x,y)

In [ ]:
def TWD972WGS84(twd97):
  '''
  TWD97 to WGS84
  '''
  wgs84 = TWD97.towgs84(twd97[0],twd97[1])
  return wgs84

In [ ]:
def createTable(table):
  '''
  create table
  '''
  if not table.table_exists():
    table.create_table()

In [ ]:
# init database
db = SqliteDatabase('company.db')
class Company(Model):
  '''
  define datatable column
  '''
  name = CharField()
  address = TextField()
  wgs84_lat = FloatField()
  wgs84_lon = FloatField()

  class Meta:
    database = db

In [ ]:
def uploadFile(filename):
  '''
  upload file
  '''
  if not os.path.exists(filename):
    uploadfile = files.upload()

In [ ]:
def go2log(log_path,data):
	'''
  write error data to log
	'''
	with open(log_path,'a', newline='') as f:
		f.write('name:{},addr:{}'.format(data[0],data[1]))


In [ ]:
if __name__ == "__main__":
    # upload file
    filename = '公司登記(依營業項目別)－機械批發業.csv'
    uploadFile(filename)
    # create table
    createTable(Company)
    # read file
    gui_list = getFile(filename)
    # init selenium (can add proxy)
    driver = initSelenium(proxy=None)
    for company in gui_list:
      # get company address
      driver,address = getAddr(driver,gui_list[company])
      # search TWD97 use address
      try:
        driver,twd97 = getTWD97(driver,address)
      except:
        #有可能地址打錯或是不明確，經查詢有複數以上地址需人工確認
        print('addr_ERR:',address)
        # add error data to log
        go2log('./err_addr.log',(company,address))
        continue
      # twd97 to wgs84
      wgs84 = TWD972WGS84(twd97)
      # insert sql
      data = Company(name=company,address=address,wgs84_lat=wgs84[0],wgs84_lon=wgs84[1])
      data.save()
      print(company,address,wgs84,'saved')

    # close selenium driver
    driver.quit()

Saving 公司登記(依營業項目別)－機械批發業.csv to 公司登記(依營業項目別)－機械批發業.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


鎮達有限公司 基隆市仁愛區南榮路57號 (25.126294967939668, 121.74272836678509) saved
青山有限公司 基隆市信義區義四路14號 (25.129719837774278, 121.74833119912049) saved
addr_ERR: 基隆市信義區深奧坑路166之14號8樓
新三洋企業有限公司 基隆市中正區港通里中正路12號3樓 (25.13239767869918, 121.74436090722735) saved
addr_ERR: 基隆市仁愛區忠二路4號2樓
民昌企業有限公司 臺北市中山區吉林路369號6樓之1 (25.06506077135248, 121.53048368909185) saved
鴻建貿易股份有限公司 基隆市中正區港通里義一路14號 (25.131101603476495, 121.74463760645908) saved
addr_ERR: 基隆市仁愛區孝四路28號
三奇漁業股份有限公司 基隆市中正區中正里中正路638之1號 (25.150930391471935, 121.76689944119266) saved
九重機械工業股份有限公司 基隆市安樂區武訓街57號 (25.14813577741494, 121.69915614911869) saved
宏駿機電工程有限公司 基隆市中正區信五路5號1樓 (25.133622068768265, 121.74805306258884) saved
正健機械工程有限公司 基隆市暖暖區源遠路38號5樓 (25.102719287332242, 121.73825430551632) saved
簡隆企業有限公司 新北市汐止區福德一路342巷3弄16號(2樓) (25.063347718690057, 121.62834990151751) saved
昇功企業有限公司 新北市金山區美田里中山路191號1樓 (25.222962562468854, 121.63651924038504) saved
玉坤機械工程有限公司 基隆市仁愛區書院里華六街21號8樓 (25.12500805442971, 121.7361105802118) saved
友亦企業股份有限公司 臺北市中山區德惠街9號6樓 (25.066798635028317,

MaxRetryError: ignored

In [ ]:
# print all data in database table
for company in Company.select():
    print(company.name,company.address,company.wgs84_lat,company.wgs84_lon)

鎮達有限公司 基隆市仁愛區南榮路57號 25.126294967939668 121.74272836678509
青山有限公司 基隆市信義區義四路14號 25.129719837774278 121.74833119912049
新三洋企業有限公司 基隆市中正區港通里中正路12號3樓 25.13239767869918 121.74436090722735
民昌企業有限公司 臺北市中山區吉林路369號6樓之1 25.06506077135248 121.53048368909185
鴻建貿易股份有限公司 基隆市中正區港通里義一路14號 25.131101603476495 121.74463760645908
三奇漁業股份有限公司 基隆市中正區中正里中正路638之1號 25.150930391471935 121.76689944119266
九重機械工業股份有限公司 基隆市安樂區武訓街57號 25.14813577741494 121.69915614911869
宏駿機電工程有限公司 基隆市中正區信五路5號1樓 25.133622068768265 121.74805306258884
正健機械工程有限公司 基隆市暖暖區源遠路38號5樓 25.102719287332242 121.73825430551632
簡隆企業有限公司 新北市汐止區福德一路342巷3弄16號(2樓) 25.063347718690057 121.62834990151751
昇功企業有限公司 新北市金山區美田里中山路191號1樓 25.222962562468854 121.63651924038504
玉坤機械工程有限公司 基隆市仁愛區書院里華六街21號8樓 25.12500805442971 121.7361105802118
友亦企業股份有限公司 臺北市中山區德惠街9號6樓 25.066798635028317 121.52464003832266
